In [9]:
import sys, random
print(sys.version_info)
seed = 33
random.seed(seed)

sys.version_info(major=2, minor=7, micro=10, releaselevel='final', serial=0)


In [3]:
import sklearn as sk
import pandas as pd
import numpy as np
import keras as ks

Using Theano backend.


In [7]:
## Load and clean the Iris data

from sklearn import datasets

iris = datasets.load_iris()

X = iris.data
y = iris.target

df = pd.DataFrame(data = np.column_stack((X, y)))
df.columns = ["Sepal length", "Sepal width", "Petal length", "Petal width", "Species"]
df["Species"] = df["Species"].astype('category')
df["Species"].cat.categories = iris.target_names

In [16]:
df.head()

,Sepal length,Sepal width,Petal length,Petal width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


The cannonical problem here is to predict the species given the phenotypic measurements. I'll use some traditional machne learning methods to do this, inluding neural networks, random forests, and a support vector machine. I'll tune  hyperparameters using cross validation in the support vector machine and out of bag error in the random forest. Finally, I'll provide standard performance metrics for each method to compare their efficacy.

In [31]:
### Let's start with the SVM

## Imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

## Parameters
test_size = 0.3
cv = 5 # number of folds in cv-fold cross validation

## Create split train and test sets
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(
    X, y, test_size=test_size, random_state=seed)

## Define parameter grid to search over
param_grid = [
    {'kernel': ['rbf'], 'gamma': np.logspace(-4, -2, 3), 'C': np.logspace(0, 3, 4)},
    {'kernel': ['linear'], 'C': np.logspace(0, 3, 4)}
]

## Find the best hyperparameters and fit the model using them
gcv_svm = GridSearchCV(SVC(probability=True), param_grid, cv=cv)
gcv_svm.fit(X_train_svm, y_train_svm) # There are only 150 observations so this is instantaneous

svm = gcv_svm.best_estimator_

## Sources
# http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html

In [27]:
### Continuing with the random forest

## Imports
from sklearn.ensemble import RandomForestClassifier

## Parameters
test_size = 0.3
nest = 7 # number of decision trees to use in the boosted classifier

## Create split train and test sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X, y, test_size=test_size, random_state=seed)

## Use out of bag error to train the random forest
rf = RandomForestClassifier(n_estimators=nest)
_ = rf.fit(X_train_rf, y_train_rf) # Using out of bag error

## Sources
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [28]:
### Finally, I'll train a nueral network

## Imports
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical

## Parameters
test_size = 0.3

## Create split train and test sets
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(
    X, y, test_size=test_size, random_state=seed)

## Create a shallow, simple architecture for the neural network
nn = Sequential()
nn.add(Dense(3, input_dim=np.shape(X)[1], activation='sigmoid'))
nn.add(Dense(1, activation='sigmoid'))
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the model with backpropagation
_ = nn.fit(X_train_nn, y_train_nn, epochs=10, batch_size=32, verbose=0)

## Sources
# http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

Now that I've fit the three canonical models, I'll gauge their performance using two standard metrics: ROC and precision-recall, acknowledging the omission of confusion matricies.

In [35]:
from sklearn.metrics import classification_report, precision_recall_curve, roc_curve

models = [svm, rf, nn]
names = ["SVM", "RF", "NN"]
test_xs = [X_test_svm, X_test_rf, X_test_nn]
test_ys = [y_test_svm, y_test_rf, y_test_nn]

conf_mxs = []
df_pr = pd.DataFrame(columns=["Strong Learner", "pre", "rec"])
df_roc = pd.DataFrame(columns=["Strong Learner", "tpr", "fpr"])

for model, name, xs, ys in zip(models, names, test_xs, test_ys):
    
    probs = model.predict_proba(xs)
    print(ys)
    prob1 = probs[:,1]

    ## Calculate ROC, Precis-Recall, and conf mx for each classifier
    fpr, tpr, _ = roc_curve(ys, prob1, pos_label=1)
    pre, rec, _ = precision_recall_curve(ys, prob1, pos_label=1)
    
    data_pr = {"Strong Learner": np.repeat(name, len(pre)),
               "pre": pre,
               "rec": rec}
    data_roc = {"Strong Learner": np.repeat(name, len(fpr)),
               "fpr": fpr,
               "tpr": tpr}
    
    df_pr = df_pr.append(pd.DataFrame(data=data_pr))
    df_roc = df_roc.append(pd.DataFrame(data=data_roc))

[1 1 0 1 2 2 0 0 2 2 2 0 2 1 2 1 2 0 1 2 0 0 2 0 2 2 1 1 2 2 1 1 2 2 2 2 2
 1 1 0 1 1 1 0 0]
[1 1 0 1 2 2 0 0 2 2 2 0 2 1 2 1 2 0 1 2 0 0 2 0 2 2 1 1 2 2 1 1 2 2 2 2 2
 1 1 0 1 1 1 0 0]
32/45 [====================>.........] - ETA: 0s[1 1 0 1 2 2 0 0 2 2 2 0 2 1 2 1 2 0 1 2 0 0 2 0 2 2 1 1 2 2 1 1 2 2 2 2 2
 1 1 0 1 1 1 0 0]


IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
from ggplot import *

## Plot PR, ROC, CMs
plt_roc = ggplot(aes(x='fpr',y='tpr',color='Strong Learner'), data=df_roc) +\
    geom_line() +\
    labs(x='False positive rate', y='True positive rate', title='ROC curves for multiple methods')
plt_roc.show()

plt_pr = ggplot(aes(x='rec',y='pre',color='Strong Learner'), data=df_pr) +\
    geom_line() +\
    labs(x='Recall', y='Precision', title='Precision-Recall curves for multiple methods')
plt_pr.show()